## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
<br>
<center>ФБ-22 Дажук Павло</center>

#### Хід виконання

In [482]:
import os
import pandas as pd
import urllib
from datetime import datetime
import re
print("Setup Complete")

Setup Complete


In [483]:
def download_csv():
    regions = {}
    ranges = [(1, 12), (13, 20), (21, 28)]
    ids = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    id = 0
    
    for start, end in ranges:
        for i in range(start, end):
            id = ids[i]
            url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2020&type=Mean'.format(i)
            wp = urllib.request.urlopen(url)
            text = wp.read()
            
            text = text.replace(b"<br>", b"")
            text = text.replace(b"<tt><pre>", b"")
            text = text.replace(b"</pre></tt>", b"")
            
            match = re.search(br':\s*([^\s,]+)', text)
            regions[id] = match.group(1).decode('utf-8')
        
            now = datetime.now()
            date_and_time_time = now.strftime("%d-%m-%Y_%H-%M-%S")
            out=open(f'data\\obl_{id}_{date_and_time_time}.csv', 'wb')
            out.write(text)
            out.close()
            print (f"{regions[id]}: {id}, ", end="")
    return regions

In [484]:
regions = download_csv()

Cherkasy: 22, Chernihiv: 24, Chernivtsi: 23, Crimea: 25, Dnipropetrovs'k: 3, Donets'k: 4, Ivano-Frankivs'k: 8, Kharkiv: 19, Kherson: 20, Khmel'nyts'kyy: 21, Kiev: 9, Kirovohrad: 10, Luhans'k: 11, L'viv: 12, Mykolayiv: 13, Odessa: 14, Poltava: 15, Rivne: 16, Sumy: 17, Ternopil': 18, Transcarpathia: 6, Vinnytsya: 1, Volyn: 2, Zaporizhzhya: 7, Zhytomyr: 5, 

In [485]:
def handling_values(folder_path):
    dfs = []
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Region_ID']
    
    file_list = os.listdir(folder_path)
    
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, header=1, names=headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        region_id = re.search(r'_(\d+)_', file_name).group(1)
        df['Region_ID'] = int(region_id)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

In [486]:
df = handling_values('data')
df.to_csv('df.csv')

In [487]:
def analyze_vhi_for_year(df, regions, region, year):
    vhi_series = df[(df['Year'] == year) & (df['Region_ID'] == region)]['VHI']
    min_vhi = vhi_series.min()
    max_vhi = vhi_series.max()
    
    print(f"{regions[region]} in {year}:")
    print(f"Min VHI: {min_vhi}")
    print(f"Max VHI: {max_vhi}")


In [488]:
analyze_vhi_for_year(df, regions, 2, 2000)

Volyn in 2000:
Min VHI: 24.65
Max VHI: 68.85


In [489]:
def analyze_vhi_for_years_and_regions(df, regions, start_year, end_year, selected_regions):
    for region in selected_regions:
        print(f"Region: {regions[region]}")
        for year in range(start_year, end_year + 1):
            vhi_series = df[(df['Year'] == year) & (df['Region_ID'] == region)]['VHI']
            avg_vhi = vhi_series.mean()
            print(f"Year: {year}, Average VHI: {round(avg_vhi, 2)}")
        print("=" * 30)


In [490]:
analyze_vhi_for_years_and_regions(df, regions, 2000, 2003, [1, 2, 3])

Region: Vinnytsya
Year: 2000, Average VHI: 40.98
Year: 2001, Average VHI: 54.1
Year: 2002, Average VHI: 52.82
Year: 2003, Average VHI: 48.81
Region: Volyn
Year: 2000, Average VHI: 50.06
Year: 2001, Average VHI: 57.97
Year: 2002, Average VHI: 51.17
Year: 2003, Average VHI: 51.32
Region: Dnipropetrovs'k
Year: 2000, Average VHI: 44.0
Year: 2001, Average VHI: 53.28
Year: 2002, Average VHI: 50.95
Year: 2003, Average VHI: 53.64


In [491]:
# Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей
def find_extreme_drought_years(df, regions, percentage):
    total_regions = len(regions)
    affected_years = []

    for year in range(1982, 2021):
        affected_regions = 0

        for region in range(1, total_regions + 1):
            min_vhi = df[(df['Year'] == year) & (df['Region_ID'] == region)]['VHI'].min()

            if min_vhi < 15:
                affected_regions += 1

        if affected_regions/total_regions > percentage / 100: 
            affected_years.append(year)
    
    print(f"Years with extreme drought in more than {percentage}% of regions: {affected_years}")


In [492]:
find_extreme_drought_years(df, regions, 5)

Years with extreme drought in more than 5% of regions: [1986, 1993, 2000, 2007]


In [493]:
def find_moderate_drought_years(df, regions, percentage):
    total_regions = len(regions)
    affected_years = []

    for year in range(1982, 2021):
        affected_regions = 0

        for region in range(1, total_regions + 1):
            vhi = df[(df['Year'] == year) & (df['Region_ID'] == region) & (df['VHI'] >= 15)]['VHI'].min()

            if vhi < 35:
                affected_regions += 1

        if affected_regions/total_regions > percentage / 100: 
            affected_years.append(year)
    
    print(f"Years with moderate drought in more than {percentage}% of regions: {affected_years}")


In [494]:
find_moderate_drought_years(df, regions, 80)

Years with moderate drought in more than 80% of regions: [1982, 1983, 1984, 1985, 1986, 1988, 1989, 1990, 1991, 1992, 1993, 1995, 1996, 2000, 2015, 2019]
